In [29]:
import pandas as pd
import numpy as np
import random
import math
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [30]:
df = pd.read_csv("tourism_with_id.csv", encoding='ISO-8859-1')
df = df[df["City"] == "Jakarta"]
price_values = df['Price(RPH)'].tolist()

for i in range(len(price_values)):
  converted_amount = price_values[i] * 0.0054
  df['Price(RPH)'][i] = math.ceil(converted_amount)

df.rename(columns={'Price(RPH)': 'Price (INR)'}, inplace=True)

category_avg = df.groupby('Categories')['Time_Minutes'].mean()

# Replace NaN values with the average time taken for each category
df['Time_Minutes'] = df.apply(lambda row: category_avg[row['Categories']] if np.isnan(row['Time_Minutes']) else row['Time_Minutes'], axis=1)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df.loc[df['Categories'] == 'Islands', 'Time_Minutes'] = df.loc[df['Categories'] == 'Islands', 'Time_Minutes'].fillna(90)
df.loc[df['Categories'] == 'Religious', 'Time_Minutes'] = df.loc[df['Categories'] == 'Religious', 'Time_Minutes'].fillna(45)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df['Time_Minutes'] = df['Time_Minutes'].apply(math.ceil)
display(df)

final_df = df

<ipython-input-30-1b796dfcd7eb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price(RPH)'][i] = math.ceil(converted_amount)


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      84 non-null     int64  
 1   Place_Name    84 non-null     object 
 2   Description   84 non-null     object 
 3   Categories    84 non-null     object 
 4   City          84 non-null     object 
 5   Price (INR)   84 non-null     int64  
 6   Ratings       84 non-null     float64
 7   Time_Minutes  84 non-null     int64  
 8   Coordinate    84 non-null     object 
 9   lat           84 non-null     float64
 10  Long          84 non-null     float64
dtypes: float64(3), int64(3), object(5)
memory usage: 9.9+ KB


In [32]:
types = []
for i in df['Categories']:
  if i not in types:
    types.append(i)
  else:
    continue

print(types)

['Culture', 'Amusement park', 'Nature preserve', 'Islands', 'Shopping center', 'Religious']


In [33]:
max_budget = int(input("Enter the maximum budget of the trip: "))
no_days = int(input("Enter the number of days of sightseeing: "))

Enter the maximum budget of the trip: 30000
Enter the number of days of sightseeing: 4


In [34]:
hotel_df = pd.read_csv("hotels_jak.csv", encoding='ISO-8859-1')
hotel_df = hotel_df.drop_duplicates()
hotel_list = hotel_df['Name'].tolist()

def extract_coordinates_from_url(url):
    pattern = r"/data=!.*!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)"
    match = re.search(pattern, url)

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return latitude, longitude
    else:
        return None, None

display(hotel_df)

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude
0,https://www.google.com/maps/place/Best+Western...,Best Western Senayan,4.5,3-star hotel,Casual hotel with dining & free Wi-Fi,2811,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.228065,106.798835
1,https://www.google.com/maps/place/Mercure+Jaka...,Mercure Jakarta Gatot Subroto,4.6,4-star hotel,Upmarket hotel with dining & a spa,4535,Pool,Free parking,Free Wi-Fi,Air-conditioned,-6.239528,106.830420
2,https://www.google.com/maps/place/Ashley+Wahid...,Ashley Wahid Hasyim Jakarta,4.5,4-star hotel,Chic lodging with a modern restaurant,3918,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.187497,106.827085
3,https://www.google.com/maps/place/HARRIS+Vertu...,HARRIS Vertu Hotel Harmoni Jakarta,4.6,4-star hotel,Cosmopolitan lodging with a spa,4007,Pool,Spa,Free parking,Free Wi-Fi,-6.164287,106.820651
4,https://www.google.com/maps/place/Hotel+Rivoli...,Hotel Rivoli Jakarta,4.2,3-star hotel,Straightforward hotel with dining,1785,Spa,Free parking,Free breakfast,Free Wi-Fi,-6.183943,106.844287
...,...,...,...,...,...,...,...,...,...,...,...,...
73,https://www.google.com/maps/place/Four+Points+...,"Four Points by Sheraton Jakarta, Thamrin",4.3,4-star hotel,Modern hotel with casual dining & a gym,4863,Spa,Free parking,Free Wi-Fi,Air-conditioned,-6.188306,106.822639
74,https://www.google.com/maps/place/K-Style+Eco+...,K-Style Eco Hotel Jakarta,3.9,2-star hotel,Modest budget hotel with a restaurant,862,Free parking,Air-conditioned,Wi-Fi,Restaurant,-6.159590,106.839515
75,https://www.google.com/maps/place/Hotel+Parago...,Hotel Paragon,4.0,3-star hotel,Casual lodging with a restaurant,1496,Free parking,Air-conditioned,Breakfast,Wi-Fi,-6.186664,106.831223
76,https://www.google.com/maps/place/THE+1O1+Jaka...,THE 1O1 Jakarta Sedayu Darmawangsa,4.5,4-star hotel,Striking hotel featuring city views,4632,Pool,Spa,Free parking,Free Wi-Fi,-6.253952,106.801820


In [35]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   ans = np.round(res, 2)
   return ans

def objective_function(solution, locations):
    total_distance = 0

    # Calculate the total distance for the solution
    for i in range(len(solution) - 1):
        current_location = solution[i]
        next_location = solution[i + 1]

        # Calculate the distance between locations
        distance = haversine_distance(locations[current_location], locations[next_location])
        total_distance += distance

    # Return the total distance
    return total_distance


In [36]:
df1 = df[['Place_Name', 'lat', 'Long']]
df1.head(10)

,Place_Name,lat,Long
0,National Monument,-6.175392,106.827153
1,Old town,-6.137645,106.817125
2,Fantasy world,-6.125312,106.833538
3,Beautiful Indonesia in Miniature Park (TMII),-6.302446,106.895156
4,Atlantis Water Adventure,-6.124190,106.839134
5,Jaya Ancol Dreamland,-6.117333,106.857995
6,Ragunan Zoo,-6.312459,106.820187
7,Ocean Ecopark,-6.125802,106.836325
8,Harbor Marina,1.078880,103.931398
9,Tidung Island,-5.803205,106.523791


In [37]:
sorted_df = df[df['Ratings'] > 4]
new_df = sorted_df

categories = new_df['Categories'].unique()
print(categories)

user_categories = []
i=0
while i < len(categories):
  choice = input("Enter the categories you wish to visit (enter 0 to stop; enter 'all' to choose all categories): ")
  if choice == 'all':
    user_categories = categories
    break

  if (choice == '0'):
    break

  choice = choice.capitalize()
  if choice in categories:
    user_categories.append(choice)
    i=i+1

  else:
    print("Invalid entry")

print(user_categories)

user_categories_df = new_df[new_df['Categories'].isin(user_categories)]
df = user_categories_df
display(df)



['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']
Enter the categories you wish to visit (enter 0 to stop; enter 'all' to choose all categories): all
['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [38]:
max_tour_duration = 650  # Maximum tour duration in minutes
min_rating = 4.0
all_tours = []
TOURS = []

def get_location(place):
      lat = df.loc[df['Place_Name'] == place, 'lat'].item()
      long = df.loc[df['Place_Name'] == place, 'Long'].item()
      return lat, long

# Calculate the commute time based on the distance
def calculate_commute_time(distance):
      # Assuming a constant speed for commute
      speed = 30  # km/h
      commute_time = distance / speed  # in hours
      return (commute_time*60)

# Get the time spent at a place
def get_time_spent(place):
      time_spent = df.loc[df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

# Get the cost of visiting a place
def get_cost(place):
      cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].item()
      return cost

# Get the rating of a place
def get_rating(place):
      rating = df.loc[df['Place_Name'] == place, 'Ratings'].item()
      return rating

# Calculate the total commute time in the tour
def calculate_total_commute_time(tour):
      total_commute_time = 0
      for i in range(len(tour)):
          place1 = tour[i]
          place2 = tour[(i + 1) % len(tour)]
          lat1, long1 = get_location(place1)
          lat2, long2 = get_location(place2)
          dist = haversine_distance(lat1, long1, lat2, long2)
          commute_time = calculate_commute_time(dist)
          total_commute_time += commute_time
      return total_commute_time



for i in range(no_days):
  duration = []
  def initialize_population(num_ants, max_tour_duration, max_budget, min_rating):
    population = []
    attractions = df['Place_Name'].tolist()
    for _ in range(num_ants):
      tour = generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)
      population.append(tour)


    return population


  def generate_random_tour(attractions, max_tour_duration, max_budget, min_rating):
    attractions = list(attractions)
    random.shuffle(attractions)
    tour = []
    distances = []
    tour_duration = 0

    if len(attractions) <= 1:
        return tour, distances

    for i in range(len(attractions)):
        attraction1 = attractions[i]

        if i < len(attractions) - 1:
            attraction2 = attractions[i + 1]

            lat1, long1 = get_location(attraction1)
            lat2, long2 = get_location(attraction2)
            dist = haversine_distance(lat1, long1, lat2, long2)

            commute_time = calculate_commute_time(dist)
            time_spent = get_time_spent(attraction2)

            # Check if adding the current attraction exceeds the max tour duration
            if tour_duration + commute_time + time_spent <= max_tour_duration:
                distances.append([attraction1, attraction2, dist])
                tour.append(attraction1)
                tour_duration += commute_time + time_spent
            else:
                # If adding the current attraction exceeds the max tour duration,
                # calculate the remaining time available
                remaining_time = max_tour_duration - tour_duration

                # Check if the remaining time is enough for the commute time and time spent at the last attraction
                if remaining_time >= (commute_time + get_time_spent(attractions[-1])):
                    # Add the last attraction to complete the tour
                    distances.append([attractions[-1], tour[0], dist])
                    tour.append(attractions[-1])
                    tour_duration += commute_time + get_time_spent(attractions[-1])
                    break  # Exit the loop since the tour is complete
                else:
                    break  # Exit the loop since the remaining time is not enough

    # If the generated tour has less than 3 places, reshuffle and regenerate the tour

    if len(tour) < 3:
        return generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)

    tour_duration = math.ceil(tour_duration)
    duration.append(tour_duration)

    return tour, distances

  tours = initialize_population(20, max_tour_duration, max_budget, min_rating)


  def calculate_tour_distance(ants):
    distances = []
    tour_list = []
    tour_distances = []
    for tour in ants:
      dists = tour[1]
      travelled_distance = 0
      for distance in dists:
        travelled_distance = travelled_distance + distance[2]
      distances.append(travelled_distance)
      tour_list.append(tour[0])

    tour_distances = [[x, y] for x, y in zip(tour_list, distances)]

    return tour_distances

  population_distances = calculate_tour_distance(tours)


  for i, j in zip(population_distances, duration):
    sum_ratings = 0
    average_rating = 0
    total_cost = 0
    for place in i[0]:
      rating_place = df.loc[df['Place_Name'] == place, 'Ratings'].iloc[0]
      cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].iloc[0]
      sum_ratings = sum_ratings + rating_place
      total_cost = total_cost + cost

    average_rating = (sum_ratings) / len(i[0])
    i.append(round(average_rating, 2))
    i.append(total_cost)

    i.append(j)


  def evaluate_fitness(tours, max_tour_duration, max_budget, min_rating):
    fitness_values = []
    for tour, total_distance, total_rating, total_budget, total_duration in tours:
      if (total_duration <= max_tour_duration and total_budget <= max_budget and total_rating >= min_rating):
        fitness = calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating)
        fitness_values.append(fitness)
      else:
        fitness_values.append(0)  # Penalize tours violating the constraints
    return fitness_values


  def calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating):
    weight_distance = 1
    weight_duration = 3
    weight_budget = 2
    weight_rating = 4

    # Check if the budget constraint is violated
    if total_budget > max_budget:
      return 0  # Penalize the solution with a fitness value of 0

    # Check if the rating constraint is violated
    if total_rating < min_rating:
      return 0  # Penalize the solution with a fitness value of 0

    fitness = (
        weight_distance * (1 / total_distance)
        + weight_duration * (1 / total_duration)
        + weight_budget * ((max_budget - total_budget) / max_budget)  # Scale the budget fitness based on how close it is to the maximum budget
        + weight_rating * (total_rating - min_rating)  # Scale the rating fitness based on how far it is from the minimum rating
    )

    return fitness

  fitness = evaluate_fitness(population_distances, max_tour_duration, max_budget, min_rating)
  fitness_values = fitness

  pheromone_constant = 1.0
  evaporation_rate = 0.5
  alpha = 1.0
  beta = 2.0

  # Tours data (places, distance, rating, number of places)
  tours = population_distances

  # ACO initialization
  num_ants = 5
  num_iterations = 10
  pheromone_levels = [pheromone_constant] * len(tours)

  # ACO algorithm
  for iteration in range(num_iterations):
      ant_tours = []
      for ant in range(num_ants):
          tour = []
          remaining_budget = max_budget

          while remaining_budget >= 0:
              available_tours = [t for t in tours if t not in tour]

              if len(available_tours) == 0:
                  break

              attractiveness_values = []

              for available_tour in available_tours:
                  tour_places, tour_distance, tour_rating, tour_cost, tour_duration = available_tour

                  attractiveness = pheromone_levels[tours.index(available_tour)] ** alpha * (tour_rating ** beta) / (tour_distance * tour_cost * tour_duration)
                  attractiveness_values.append(attractiveness)

              total_attractiveness = sum(attractiveness_values)

              if total_attractiveness == 0:
                  next_tour = random.choice(available_tours)
              else:
                  normalized_probabilities = [a / total_attractiveness for a in attractiveness_values]
                  cumulative_probabilities = [sum(normalized_probabilities[:i+1]) for i in range(len(normalized_probabilities))]
                  random_value = random.uniform(0, 1)
                  next_tour_index = next((i for i, cp in enumerate(cumulative_probabilities) if cp >= random_value), len(cumulative_probabilities) - 1)
                  next_tour = available_tours[next_tour_index]

              tour.append(next_tour)
              remaining_budget -= next_tour[3]  # Budget at index 3

          ant_tours.append(tour)

      for ant_tour in ant_tours:
        tour_ratings = []
        tour_budgets = []
        tour_distances = []
        num_places_list = []

        tour_ratings = [tour[2] for tour in tours]
        average_rating = sum(tour_ratings) / len(tour_ratings)
        max_rating = max(tour_ratings)
        closest_to_max_budget = max([tour[3] for tour in tours]) - max_budget
        shortest_distance = min([tour[1] for tour in tours])
        max_num_places = max([len(tour[0]) for tour in tours])

        for tour in ant_tour:
            tour_index = tours.index(tour)
            pheromone_levels[tour_index] *= evaporation_rate  # Apply pheromone evaporation
            tour_places, tour_distance, tour_rating, tour_budget, tour_duration = tour

            # Update pheromone levels based on the attractiveness and constraints
            if tour_budget <= max_budget:
                pheromone_levels[tour_index] += (1.0 / average_rating) + (1.0 / max_rating) + (1.0 / closest_to_max_budget) + (1.0 / shortest_distance)


  best_tour = None
  best_score = float('inf')

  weight_distance = 1
  weight_duration = 3
  weight_budget = 2
  weight_rating = 4

  for tour in tours:
      tour_places, tour_distance, tour_rating, tour_budget, tour_duration = tour

      # Check if the tour satisfies the budget constraint
      if tour_budget <= max_budget and tour_duration <= max_tour_duration:
          score = (weight_distance * tour_distance) + (weight_rating * tour_rating) + (weight_budget * tour_budget)

          # Update the best tour if the current tour has a better score
          if score < best_score:
              best_tour = tour
              best_score = score

  # Print the information of the best tour
  all_tours.append(best_tour)
  if best_tour is not None:
      tour_places, tour_distance, tour_rating, tour_budget, tour_duration = best_tour
      print("Best Tour: ", tour_places)

  else:
      print("No tour satisfies the budget constraint.")

  df = df[~df["Place_Name"].isin(best_tour[0])]

my_tours = []
for i in all_tours:
  my_tours.append(i[0])





Best Tour:  ['Suropati Park', 'Museum of Fine Arts and Ceramics', 'National Library']
Best Tour:  ['Set Babakan', 'Menteng Park', 'Welcome Monument', 'Nine Plot Market', 'Indonesian Bank Museum']
Best Tour:  ['Central Garden Museum', 'Diamond City Bridge', 'Puppet Museum']
Best Tour:  ['Cathedral Church', 'Spathodea Park', 'Istiqlal Mosque', 'Freedom Libraries', 'Grand Indonesia Mall', 'Situ Lembang Park']


In [39]:
cost = 0
for i in all_tours:
  cost = cost + (7000*0.0054) + (i[1]-2)*4000*0.0054

for i in my_tours:
  for j in i:
    price = final_df.loc[final_df['Place_Name'] == j, 'Price (INR)'].iloc[0]
    cost = cost + price

cost_tours = math.ceil(cost)



In [40]:
budget_constraint = max_budget - cost_tours - 500

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        place = item[0]
        if place not in seen:
            seen.add(place)
            result.append(item)
    return result

def calculate_score(distance, rating, price_per_night, total_nights, budget):
    weight_distance = 2
    weight_rating = 8
    weight_budget = 3
    total_price = price_per_night * total_nights
    budget_difference = abs(budget - total_price)
    score = (weight_distance / (distance + 0.1)) + (weight_rating * rating) + (weight_budget / (budget_difference + 0.1))
    return score

hotels = hotel_df['Name'].tolist()
hotel_ratings = hotel_df['Rating'].tolist()
hotel_prices_per_night = hotel_df['Price per Night'].tolist()
total_nights = no_days + 2  # Example number of total nights, adjust according to your requirements

hotel_scores = []
for tour in my_tours:
    first_place = tour[0]
    first_place_lat = final_df.loc[final_df['Place_Name'] == first_place, 'lat'].iloc[0]
    first_place_long = final_df.loc[final_df['Place_Name'] == first_place, 'Long'].iloc[0]

    for hotel, rating, price_per_night in zip(hotels, hotel_ratings, hotel_prices_per_night):
        url = hotel_df.loc[hotel_df['Name'] == hotel, 'Maps'].iloc[0]
        hotel_latitude, hotel_longitude = extract_coordinates_from_url(url)
        distance = haversine_distance(first_place_lat, first_place_long, hotel_latitude, hotel_longitude)
        total_price = price_per_night * total_nights
        if total_price <= budget_constraint:
            score = calculate_score(distance, rating, price_per_night, total_nights, budget_constraint)
            hotel_scores.append((hotel, rating, distance, price_per_night, score))

hotel_scores.sort(key=lambda x: (x[1], abs(x[3] * total_nights - budget_constraint), x[2]), reverse=True)  # Sort by rating (descending), total price difference (ascending), and distance (ascending)
hotel_scores = remove_duplicates(hotel_scores)

top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]
print(top_hotels)




['Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel', 'Hotel Ciputra Jakarta (Managed by Swiss-Belhotel International)', 'Hotel Des Indes, Menteng', 'Erian Hotel', 'Stanley Wahid Hasyim Jakarta', 'HARRIS Vertu Hotel Harmoni Jakarta', 'YELLO Hotel Manggarai - Jakarta', 'YELLO Harmoni Jakarta', 'Swiss-Belinn Wahid Hasyim', 'Liberty Hotel Thamrin Jakarta', 'Best Western Senayan', 'ARTOTEL Thamrin - Jakarta', 'Jambuluwuk Thamrin Hotel', 'Ashley Sabang Jakarta', 'Ashley Wahid Hasyim Jakarta', 'Aryaduta Menteng', 'Mercure Jakarta Sabang', 'Sofyan Hotel Cut Meutia Cikini Menteng Jakarta', 'Sentral Cawang Hotel Jakarta', 'Ana Hotel', 'Grand Thamrin Hotel', 'ARTOTEL Wahid Hasyim - Jakarta', 'Eightin Hotel Sudirman Jakarta', 'Sotis Hotel Kemang Jakarta', 'Royal Kuningan Hotel', 'Best Western Premier The Hive', 'Harris Hotel Tebet - Jakarta', 'Lumire Hotel & Convention Center', 'Shakti', 'Cikini Hotel', 'PAX Hotel Jakarta', 'Hotel Marcopolo Jakarta', 'Park Hotel Jakarta', 'Ã©L Hotel Royale Ja

In [41]:
desired_order = []

for i in (top_hotels):
  index = hotel_df.index[hotel_df['Name'] == i].tolist()
  desired_order.append(index[0])

# Reorder the rows based on the desired order of row labels
hotel_df_reordered = hotel_df.reindex(desired_order)



In [42]:
top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]

dff = hotel_df_reordered

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


features = ['Rating', 'Type', 'Description', 'Price per Night', 'Amenities 1', 'Amenities 2', 'Amenities 3', 'Amenities 4']

dff['Features'] = dff[features[2:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(dff['Features'])

similarity_matrix = cosine_similarity(feature_vectors, feature_vectors)

def get_recommendations(keywords, top_n=5):

    user_input_doc = ' '.join(keywords)
    user_input_vector = vectorizer.transform([user_input_doc])

    similarity_scores = cosine_similarity(user_input_vector, feature_vectors)[0]

    top_indices = similarity_scores.argsort()[::-1][:top_n]
    top_hotels = dff.iloc[top_indices]

    top_hotels = top_hotels.sort_values(['Rating', 'Price per Night'], ascending=[False, True])

    return top_hotels['Name'].values

user_keywords = ['Free breakfast', 'Spa', 'Pool']
num_recommendations = 5
recommendations = get_recommendations(user_keywords, top_n=num_recommendations)

print("Recommendations based on distance, rating and budget:")

if len(top_hotels)>5:
  for i in range(5):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels[:5]:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)

else:
  for i in range(len(top_hotels)):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)



print()
print(f"Recommendations based on user input keywords, rating, and price per night (Top {num_recommendations} Recommendations):")
for i, hotel in enumerate(recommendations):
    print(f"{i+1}. {hotel}")




for i, hotel in enumerate(recommendations):
    recommended_hotels = dff[dff['Name'] == hotel]
    display(recommended_hotels)




Recommendations based on distance, rating and budget:
1. Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel
2. Hotel Ciputra Jakarta (Managed by Swiss-Belhotel International)
3. Hotel Des Indes, Menteng
4. Erian Hotel
5. Stanley Wahid Hasyim Jakarta


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
39,https://www.google.com/maps/place/Holiday+Inn+...,"Holiday Inn & Suites Jakarta Gajah Mada, an IH...",4.7,4-star hotel,Modern lodging with free Wi-Fi & dining,3527,Pool,Spa,Free parking,Free Wi-Fi,-6.145215,106.815086,Modern lodging with free Wi-Fi & dining 3527 P...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
37,https://www.google.com/maps/place/Hotel+Ciputr...,Hotel Ciputra Jakarta (Managed by Swiss-Belhot...,4.7,4-star hotel,Stylish hotel with a rooftop pool,3606,Pool,Spa,Free parking,Free Wi-Fi,-6.167883,106.786752,Stylish hotel with a rooftop pool 3606 Pool Sp...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
52,https://www.google.com/maps/place/Hotel+Des+In...,"Hotel Des Indes, Menteng",4.6,Hotel,Upscale lodging with dining & a bar,2601,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.197208,106.828358,Upscale lodging with dining & a bar 2601 Free ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
30,https://www.google.com/maps/place/Erian+Hotel/...,Erian Hotel,4.6,3-star hotel,Relaxed lodging with a restaurant,2842,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.187469,106.829182,Relaxed lodging with a restaurant 2842 Free pa...


...

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
24,https://www.google.com/maps/place/Sentral+Cawa...,Sentral Cawang Hotel Jakarta,4.4,3-star hotel,Contemporary hotel with dining & a pool,1754,Pool,Spa,Free parking,Free breakfast,-6.246978,106.872788,Contemporary hotel with dining & a pool 1754 P...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
25,https://www.google.com/maps/place/Best+Western...,Best Western Premier The Hive,4.4,4-star hotel,Modern lodging with a pool & dining,3074,Pool,Spa,Free parking,Free Wi-Fi,-6.244606,106.875471,Modern lodging with a pool & dining 3074 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
59,https://www.google.com/maps/place/Lumire+Hotel...,Lumire Hotel & Convention Center,4.4,4-star hotel,Polished hotel with dining & a pool,3750,Pool,Spa,Free breakfast,Free Wi-Fi,-6.175975,106.840408,Polished hotel with dining & a pool 3750 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
54,https://www.google.com/maps/place/%C3%A9L+Hote...,Ã©L Hotel Royale Jakarta,4.3,4-star hotel,Stylish lodging with a pool & a spa,2568,Pool,Spa,Free parking,Free breakfast,-6.15229,106.89624,Stylish lodging with a pool & a spa 2568 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
12,https://www.google.com/maps/place/The+Jayakart...,The Jayakarta SP Jakarta Hotel & Spa,4.2,4-star hotel,Upscale lodging with dining & a pool,1829,Pool,Spa,Free parking,Free Wi-Fi,-6.147389,106.817227,Upscale lodging with dining & a pool 1829 Pool...


In [43]:
choice = 0
my_hotel = input("Enter the hotel of your choice: ")
if my_hotel in hotel_list:
  choice = 1

while choice != 1:
  print("Invalid entry")
  my_hotel = input("Enter the hotel of your choice: ")
  if my_hotel in hotel_list:
    choice = 1


Enter the hotel of your choice: Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel


In [44]:
def get_time_spent(place):
      time_spent = final_df.loc[final_df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

def get_location(place):
      lat = final_df.loc[final_df['Place_Name'] == place, 'lat'].item()
      long = final_df.loc[final_df['Place_Name'] == place, 'Long'].item()
      return lat, long

def get_distance_from_hotel(place, my_hotel):
  url = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Maps'].item()
  hotel_lat, hotel_long = extract_coordinates_from_url(url)
  lat1, long1 = get_location(place)
  dist = haversine_distance(hotel_lat,hotel_long,lat1,long1)
  return dist

def generate_itinerary(places):
    itinerary = []
    current_time = 9 * 60  # Starting time in minutes (9 AM = 9 * 60 minutes)

    # Add hotel as the first place in the itinerary
    hotel = my_hotel

    for i in range(len(places)):
        place = places[i]
        time_spent = get_time_spent(place)

        if i == 0:
            commute_time = calculate_commute_time(get_distance_from_hotel(place, my_hotel))  # Calculate commute time from hotel
        else:
            previous_place = places[i - 1]
            lat1, long1 = get_location(previous_place)
            lat2, long2 = get_location(place)
            dist = haversine_distance(lat1, long1, lat2, long2)
            commute_time = calculate_commute_time(dist)

        start_time = current_time + commute_time
        end_time = start_time + time_spent

        itinerary.append((place, start_time, end_time))

        current_time = end_time

    return itinerary


In [45]:
TOURS = all_tours
TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost


print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")


Here's your complete itinerary: Hotel - Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel - Price per Night = Rs. 3527

Day 1 - 
Tour - ['Suropati Park', 'Museum of Fine Arts and Ceramics', 'National Library']
Total distance (Starting from and ending at hotel) - 29.14 km
Rating -  4.57
Cost - Rs. 27
Taxi cost - Rs. 625
----------------------------------
Day 2 - 
Tour - ['Set Babakan', 'Menteng Park', 'Welcome Monument', 'Nine Plot Market', 'Indonesian Bank Museum']
Total distance (Starting from and ending at hotel) - 47.45 km
Rating -  4.54
Cost - Rs. 11
Taxi cost - Rs. 1020
----------------------------------
Day 3 - 
Tour - ['Central Garden Museum', 'Diamond City Bridge', 'Puppet Museum']
Total distance (Starting from and ending at hotel) - 53.15 km
Rating -  4.47
Cost - Rs. 27
Taxi cost - Rs. 1143
----------------------------------
Day 4 - 
Tour - ['Cathedral Church', 'Spathodea Park', 'Istiqlal Mosque', 'Freedom Libraries', 'Grand Indonesia Mall', 'Situ Lembang Park']
Total dist

In [46]:
print("Detailed Itinerary:")
r = hotel_df.loc[hotel_df["Name"] == my_hotel, "Rating"].item()
print(f"Hotel: {my_hotel}, Rating: {r}")
print()
for i in all_tours:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")



Detailed Itinerary:
Hotel: Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel, Rating: 4.7

Start from hotel at 09:00
09:12 to 11:26 - Suropati Park
11:41 to 13:16 - Museum of Fine Arts and Ceramics
13:27 to 17:27 - National Library
--------------------------------------------------------------
Start from hotel at 09:00
09:43 to 11:18 - Set Babakan
11:51 to 14:05 - Menteng Park
14:06 to 14:21 - Welcome Monument
14:33 to 15:18 - Nine Plot Market
15:19 to 16:54 - Indonesian Bank Museum
--------------------------------------------------------------
Start from hotel at 09:00
09:27 to 11:27 - Central Garden Museum
11:57 to 13:32 - Diamond City Bridge
13:33 to 16:03 - Puppet Museum
--------------------------------------------------------------
Start from hotel at 09:00
09:06 to 09:51 - Cathedral Church
10:25 to 10:55 - Spathodea Park
11:29 to 12:14 - Istiqlal Mosque
12:22 to 13:57 - Freedom Libraries
14:03 to 16:03 - Grand Indonesia Mall
16:06 to 18:20 - Situ Lembang Park
----------------